In [100]:
import bs4 as bs
import urllib.request
import requests

In [128]:
source = urllib.request.urlopen('https://bbc.com')
soup= bs.BeautifulSoup( source, 'html.parser')

In [134]:
####For demo we changed disasters to other topics to show how this code
####works, but on deployment the keywords would change
####Currently, the world seems safe as per bbc.com (british news agency)
#disasters= [ 'volcano', 'volcanic eruption', 'earthquake', 'quake', 'flood', 'typhoon', 'hurricane', 'storm', 'hail', 'disaster', 'rainstorm', 'landslide', 'leakage', 'chemical leakage', 'tornado']
disasters= ['trump', 'immigration', 'diabetes']
#Volcano, Earthquake, Waterflood, Hail, Snow, Tornado, Hurricane
behaviors= ["Stay away from active volcanoes, Use goggles mask and flashlight, Drive rather than walk, Close all windows doors and fireplace, If caught in a rockfall roll into a ball to protect your head","Stay away from active volcanoes, Use goggles mask and flashlight, Drive rather than walk, Close all windows doors and fireplace, If caught in a rockfall roll into a ball to protect your head","Do not wait until you see rising water, Get out of low areas and move to high ground, Do not drive through flooded areas, Keep clear of power lines and electrical wires","Stay Indoors, If driving pull to a safe place so hail doesn't break the windshield","Keep your thermostat as high as possible for as long as you have electricity, Stay inside, Limit travel to emergencies only, Keep pipes from freezing by turning on every water faucet to a slow drip, Wear dry or waterproof clothes, Stay hydrated with plenty of fluids","Go to the basement or take shelter in a small interior ground floor room such as a bathroom closet or hallway, If you have no basement protect yourself by taking shelter under a heavy table or desk, Stay away from windows outside walls and doors","Go to the basement or take shelter in a small interior ground floor room such as a bathroom closet or hallway, If you have no basement protect yourself by taking shelter under a heavy table or desk, Stay away from windows outside walls and doors"]

In [144]:
sites= []
h3s= soup.find_all('h3')
for tag in h3s:
    for keyword in tag.getText().lower().split():
        if keyword in disasters:
            a= tag.find('a', href= True)
            if a:
                sites.append( a['href'])
h2s= soup.find_all( 'h2')
for tag in h2s:
    for keyword in tag.getText().lower().split():
        if keyword in disasters:
            a= tag.find('a', href= True)
            if a:
                sites.append( a['href'])
for i, href in enumerate( sites):
    if 'bbc.com' not in href:
        sites[ i]= ''.join( [ 'bbc.com', href])

In [142]:
h2s[0].find('a', href= True)

In [145]:
sites

['bbc.com/news/uk-48978484', 'bbc.com/news/world-us-canada-48978394']

In [146]:
headers = {'Content-Type': 'application/json',}
params = (('version', '2018-11-16'),)
responses= []
for site in sites:
    data = '{\n  "url": "' + site + '",\n  "features": {\n    "sentiment": {},\n    "categories": {},\n    "concepts": {},\n    "emotion": {},\n    "entities": {},\n    "keywords": {}\n  }\n}'
    responses.append( requests.post('https://gateway-tok.watsonplatform.net/natural-language-understanding/api/v1/analyze', headers=headers, params=params, data=data, auth=('apikey', 'rwvQDylZAfkwqNKwPqNdmiTt-Hdbd8rL7raHoJe4NMjz')))

In [82]:
responses[0].json()

{'usage': {'text_units': 1, 'text_characters': 1282, 'features': 6},
 'sentiment': {'document': {'score': -0.330014, 'label': 'negative'}},
 'retrieved_url': 'https://www.bbc.com/news/world-asia-48973515',
 'language': 'en',
 'keywords': [{'text': 'opposition leader Prabowo Subianto',
   'relevance': 0.785706,
   'count': 1},
  {'text': 'Indonesian President Joko Widodo',
   'relevance': 0.753607,
   'count': 1},
  {'text': 'opposition leader', 'relevance': 0.62362, 'count': 1},
  {'text': 'mass rapid transit', 'relevance': 0.600881, 'count': 1},
  {'text': 'opposition supporters', 'relevance': 0.590287, 'count': 1},
  {'text': 'Prabowo Subianto', 'relevance': 0.585913, 'count': 1},
  {'text': 'Mr Widodo', 'relevance': 0.585142, 'count': 1},
  {'text': 'global competition', 'relevance': 0.580699, 'count': 1},
  {'text': "Indonesia's Constitutional Court",
   'relevance': 0.571843,
   'count': 1},
  {'text': 'president', 'relevance': 0.569598, 'count': 3},
  {'text': 'fellow countrymen'

In [83]:
responses[0].json()['keywords'][0]['text']#['score']

'opposition leader Prabowo Subianto'

In [84]:
disasters

['leader', 'rival', 'witness']

In [92]:
responses[0].json()['sentiment']['document']['score']

-0.330014

In [180]:
news_feed= []
for response in responses:
    #threshold should change during deployment to -0.5
    if response.json()['sentiment']['document']['score'] < 1:
        current_news_feed= dict()
        type_of_disaster= ''
        location= []
        for i in range( 10):
            words= response.json()['keywords'][i]['text'].lower().split()
            for word in words:
                if word in disasters:
                    type_of_disaster= word
                    break
            if len(type_of_disaster) > 0:
                break
        if len( type_of_disaster) > 0:
            current_news_feed[ 'type']= type_of_disaster
            for entity in response.json()['entities']:
                if entity['type'] == 'Location':
                    for dis in entity['disambiguation']['subtype']:
                        if dis in [ 'City', 'Country'] and dis not in location:
                            location.append( dis)
                            current_news_feed[ dis]= entity[ 'text']
                if len( location) == 2:
                    break
        if len( current_news_feed) > 0:
            news_feed.append( current_news_feed)

In [148]:
news_feed

[{'type': 'immigration', 'City': 'Tacoma'}]

In [184]:
for news in news_feed:
    if news['type']== 'trump':
        info_code= 1
    elif news['type']== 'diabetes':
        info_code= 2
    else:
        info_code= 3
    if news['Country']== 'China':
        lang_code= 'zh'
    elif news['Country']== 'India':
        lang_code= 'hi'
    else:
        lang_code= 'en'
    file = open( news['City'] + ".txt", "w")
    data = '{"text": ["'+ behaviors[ info_code] +'"], "model_id":"en-es"}'
    response = requests.post('https://gateway-lon.watsonplatform.net/language-translator/api/v3/translate', headers=headers, params=params, data=data, auth=('apikey', 'eFT3phEGmpEQqmqm1pRrUQS_H7dxEBIwRfYIEBOhA7MD'))
    file.write(news['type'] + "in" + news['City'] + " " + news[ 'Country'] + "\n" + response.json()['translations']['translation']) 
    file.close() 

TypeError: list indices must be integers or slices, not str